In [1]:
! pip install import-ipynb

/bin/pip:6: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import load_entry_point


In [2]:
from model import BodyposeModel
import cv2
import numpy as np
import math
import time
from scipy.ndimage.filters import gaussian_filter
import matplotlib.pyplot as plt
import matplotlib
import torch
from torchvision import transforms
import util
from torchsummary import summary
from body import Body
import import_ipynb
from train_model import BodyposeBackbone, MainModel
from coco_dataset import COCODataset, transform_image
from copy import deepcopy

/tmp/ipykernel_30675/2162950386.py:6: DeprecationWarning: Please use `gaussian_filter` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter


importing Jupyter notebook from train_model.ipynb
importing Jupyter notebook from coco_dataset.ipynb


In [3]:
NO_OF_OUTPUTS = 21

In [4]:
backbone_model_path = '../model/body_pose_model.pth'

In [5]:
backbone_model = BodyposeBackbone()

In [6]:
backbone_params = torch.load(backbone_model_path)

In [7]:
backbone_dict = util.transfer(backbone_model, backbone_params)

In [8]:
backbone_model.load_state_dict(backbone_dict)

<All keys matched successfully>

In [9]:
backbone_model.eval()

BodyposeBackbone(
  (model): Sequential(
    (conv1_1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu_conv1_1): ReLU(inplace=True)
    (conv1_2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu_conv1_2): ReLU(inplace=True)
    (pool1_stage1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2_1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu_conv2_1): ReLU(inplace=True)
    (conv2_2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu_conv2_2): ReLU(inplace=True)
    (pool2_stage1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv3_1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu_conv3_1): ReLU(inplace=True)
    (conv3_2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu_conv3_2): ReLU(inplace=True)
    (conv3_3): Conv2d(256, 256, kernel_size

In [10]:
main_model = MainModel(NO_OF_OUTPUTS)

In [11]:
train_data = COCODataset('/media/jakub/One Touch/coco_pose/coco2017labels-pose/coco-pose', transforms=transform_image(split='train'))

loading annotations into memory...
Done (t=0.12s)
creating index...
index created!


In [12]:
for item in train_data.coco.anns.values():
    print(item['num_keypoints']) 

10
13
0
13
0
13
12
16
6
13
0
0
11
13
13
16
0
9
7
17
14
14
15
16
7
7
11
15
14
11
6
17
8
15
15
13
16
12
8
12
9
17
7
13
6
16
9
6
10
8
17
14
14
12
12
16
13
15
12
2
7
8
0
13
10
5
15
12
6
11
2
12
9
11
5
10
12
9
0
9
11
0
10
12
10
9
10
14
12
5
12
9
12
12
15
7
0
8
15
8
14
0
12
14
4
16
16
9
16
4
5
0
3
0
0
6
0
0
6
0
13
13
2
16
12
8
0
0
0
11
12
17
0
12
16
0
0
13
11
8
0
4
0
13
0
11
10
16
0
0
0
6
0
11
6
15
14
12
9
0
0
0
7
4
0
8
5
9
17
8
15
16
10
0
0
0
0
11
16
0
0
0
0
0
0
16
13
0
0
0
16
14
0
12
17
15
11
0
0
0
0
13
0
16
0
14
0
0
0
5
0
0
0
0
8
0
0
3
0
7
0
14
15
14
13
8
12
15
12
0
0
17
17
16
14
14
12
15
15
3
9
15
13
16
9
12
8
17
6
12
13
12
5
0
12
15
1
1
14
10
14
16
15
11
12
10
12
11
17
16
9
4
14
12
14
6
12
15
10
17
8
15
0
0
7
15
3
16
14
13
16
12
12
7
11
14
15
10
15
0
1
9
13
13
9
17
16
17
0
7
11
3
15
6
13
8
11
10
11
15
15
6
7
4
15
0
16
4
10
12
12
0
10
13
10
15
13
6
11
0
13
2
0
9
0
6
11
1
9
3
0
11
4
5
0
0
0
6
4
0
8
8
5
13
8
11
11
13
9
15
2
17
0
0
7
13
0
0
13
0
4
0
0
4
0
0
0
3
14
0
12
8
0
16
6
6
12
0
0
0
0

In [23]:
train_data.coco.getImgIds()[0]

397133

In [46]:
train_data.coco.anns[183126]

{'segmentation': [[125.12,
   539.69,
   140.94,
   522.43,
   100.67,
   496.54,
   84.85,
   469.21,
   73.35,
   450.52,
   104.99,
   342.65,
   168.27,
   290.88,
   179.78,
   288,
   189.84,
   286.56,
   191.28,
   260.67,
   202.79,
   240.54,
   221.48,
   237.66,
   248.81,
   243.42,
   257.44,
   256.36,
   253.12,
   262.11,
   253.12,
   275.06,
   299.15,
   233.35,
   329.35,
   207.46,
   355.24,
   206.02,
   363.87,
   206.02,
   365.3,
   210.34,
   373.93,
   221.84,
   363.87,
   226.16,
   363.87,
   237.66,
   350.92,
   237.66,
   332.22,
   234.79,
   314.97,
   249.17,
   271.82,
   313.89,
   253.12,
   326.83,
   227.24,
   352.72,
   214.29,
   357.03,
   212.85,
   372.85,
   208.54,
   395.87,
   228.67,
   414.56,
   245.93,
   421.75,
   266.07,
   424.63,
   276.13,
   437.57,
   266.07,
   450.52,
   284.76,
   464.9,
   286.2,
   479.28,
   291.96,
   489.35,
   310.65,
   512.36,
   284.76,
   549.75,
   244.49,
   522.43,
   215.73,
   546.88,
  

In [49]:
anns = []

In [50]:
for id in train_data.coco.getAnnIds():
    if train_data.coco.anns[id]['image_id'] == train_data.coco.getImgIds()[0]:
        ann = train_data.coco.anns[id]
        anns.append(ann)



In [51]:
bbox = anns[0]['bbox']

In [61]:
keypoints = anns[0]['keypoints']

In [65]:
keypoints = [[keypoints[i], keypoints[i+1], keypoints[i+2]] for i in range(0, len(keypoints), 3)]

In [57]:
bbox = [int(box) for box in bbox]

In [69]:
image = train_data._load_image(train_data.coco.getImgIds()[0])
cv2.rectangle(image, (bbox[0], bbox[1]), (bbox[0]+bbox[2], bbox[1]+bbox[3]), (0, 0, 255))
for keypoint in keypoints:
    cv2.circle(image, (keypoint[0], keypoint[1]), radius=0, color=(0, 0, 255), thickness=3)
cv2.imshow('Example image', image)
cv2.waitKey(5_000)
cv2.destroyAllWindows()

[388.66, 69.92, 109.41, 277.62]

In [12]:
def train(backbone_model, main_model, loss, optimizer, inputs, outputs):
    optimizer.zero_grad()
    main_model.train()

    out1 = backbone_model(inputs)
    logits = main_model(out1)
    output = loss.forward(logits, outputs)
    output.backward()
    optimizer.step()

    return output.item()

In [13]:
loss = torch.nn.MSELoss()
params = [p for p in main_model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params)

In [14]:
epochs = 10
batch_size = 4
num_of_batches = Xtrain_torch.size[0] // batch_size

for epoch in range(epochs):
    for batch in range(num_of_batches):
        inputs = Xtrain_torch[batch*batch_size:(batch+1)*batch_size]
        outputs = Ytrain_torch[batch*batch_size:(batch+1)*batch_size]
        output = train(backbone_model, main_model, loss, optimizer, inputs, outputs)

NameError: name 'Xtrain_torch' is not defined